In [0]:
import requests

def extract_data(date, base="BRL"):

    url = f"https://api.apilayer.com/exchangerates_data/{date}&base={base}"

    headers= {
    "apikey": "26fIuqF2C5C9BA52xsBxEF8YaAiXnddd"
    }

    parameters = {"base":base}
    

    response = requests.request("GET", url, headers=headers, params=parameters)

    if response.status_code != 200:
        raise Exception(f'Request failed with status {response.status_code}')
    return response.json()


In [0]:
extract_data("2025-07-20", base="BRL")